In [1]:
def make_kde_dataset(carrier_list, range_start, range_end, bandwidth):

        xs = []
        ys = []
        colors = []
        labels = []

        for i, carrier in enumerate(carrier_list):
            subset = flights[flights['name'] == carrier]
            subset = subset[subset['arr_delay'].between(range_start, range_end)]

            kde = gaussian_kde(subset['arr_delay'], bw_method=bandwidth)
            # Evenly space x values
            x = np.linspace(range_start, range_end, 100)
            # Evaluate pdf at every value of x
            y = kde.pdf(x)

            # Append the values to plot
            xs.append(list(x))
            ys.append(list(y))

            # Append the colors and label
            colors.append(airline_colors[i])
            labels.append(carrier)

        kernel_source = ColumnDataSource(data={'x': xs, 'y': ys, 'color': colors, 'label': labels})

        return kernel_source

In [2]:
def make_kde_plot(kde_src):
        p = figure(plot_width = 700, plot_height = 700,
                   title = 'Density Plot of Arrival Delays by Airline',
                   x_axis_label = 'Delay (min)', y_axis_label = 'Density')


        p.multi_line('x', 'y', color = 'color', legend = 'label', 
                     line_width = 3,
                     source = kde_src)

        # Hover tool with next line policy
        hover = HoverTool(tooltips=[('Carrier', '@label'), 
                                    ('Delay', '$x'),
                                    ('Density', '$y')],
                          line_policy = 'next')

        # Add the hover tool and styling
        p.add_tools(hover)

        p = style(p)

        return p

In [3]:
def update(attr, old, new):
        # List of carriers to plot
        carriers_to_plot = [carrier_selection.labels[i] for i in carrier_selection.active]
        
        # If no bandwidth is selected, use the default value
        if bandwidth_choose.active == []:
            bandwidth = None
        # If the bandwidth select is activated, use the specified bandwith
        else:
            bandwidth = bandwidth_select.value
            
        
        new_kde_src = make_kde_dataset(carriers_to_plot,
                                       range_start = range_select.value[0],
                                       range_end = range_select.value[1],
                                       bandwidth = bandwidth)
        
        kde_src.data.update(new_kde_src.data)

In [4]:
 def style(p):
        # Title 
        p.title.align = 'center'
        p.title.text_font_size = '20pt'
        p.title.text_font = 'serif'

        # Axis titles
        p.xaxis.axis_label_text_font_size = '14pt'
        p.xaxis.axis_label_text_font_style = 'bold'
        p.yaxis.axis_label_text_font_size = '14pt'
        p.yaxis.axis_label_text_font_style = 'bold'

        # Tick labels
        p.xaxis.major_label_text_font_size = '12pt'
        p.yaxis.major_label_text_font_size = '12pt'

        return p